In [ ]:
import os, json, glob
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from IPython.display import display, Markdown
import asyncio
from openai import AsyncAzureOpenAI
from configs.credentials_config import API_KEY, ENDPOINT, MODEL, DEPLOYMENT, EMBEDDINGS_API_KEY, EMBEDDINGS_ENDPOINT, EMBEDDINGS_VERSION, EMBEDDINGS_DEPLOYMENT # Crear archivo credentials_config.py con las credenciales de Azure OpenAI siguiendo el template
import torch
import requests
import numpy as np
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from embeddings_wrapper import LangchainSentenceTransformer, AzureOpenAIEmbedder
from typing import Any
from pydantic import BaseModel, ValidationError, Field

/Users/maxi/miniforge3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
VECTORSTORE_PATH = "datasets/fallos_vectorstore_ada-002"
EMBEDDINGS_OPTION = "azure_openai" # "langchain" or "azure_openai"
EMBEDDINGS_MODEL = "mixedbread-ai/mxbai-embed-large-v1"

In [ ]:
# IDEA DE COMO CARGAR VECTORSTORE: MODIFICAR EL ORIGINAL PARA QUE TENGA METADATOS PARA:
# * fuente: nombre del fallo (ej. "8104")
# * seccion: sección del fallo (ej. "Antecedentes", "Fundamentos de Derecho", etc.)

# 1. Cargar el índice FAISS desde el disco
db = FAISS.load_local(
    VECTORSTORE_PATH,
    embeddings=None,
    allow_dangerous_deserialization=True
)

# 2. Cantidad total de vectores (# chunks) 
print("Total de vectores en FAISS index:", db.index.ntotal)

# 3. Revisar cuántos Document guarda el docstore
docs_dict = db.docstore._dict
print("Total de Document en docstore:", len(docs_dict))

# 4. Inspeccionar los primeros 5 Document para ver texto y metadata
print("\n--- Primeros 5 Documentos ---")
for i, (doc_id, doc) in enumerate(docs_dict.items()):
    if i >= 5:
        break
    print(f"ID: {doc_id}")
    print("Texto (fragmento):", repr(doc.page_content[:100]) + "…")
    print("Metadatos:", doc.metadata)
    print("-" * 40)

# 5. Convertir metadata a DataFrame para análisis completo
records = []
for doc in docs_dict.values():
    # Si metadata estuviera vacío, aparecerán valores por defecto
    records.append({
        "fuente":  doc.metadata.get("fuente", None),
        "seccion": doc.metadata.get("seccion", None),
    })

df = pd.DataFrame(records)

# 5.a. Número de fallos distintos y chunks
print("\n=== Estadísticas generales ===")
summary = pd.DataFrame({
    "Total fallos únicos":  [df["fuente"].nunique(dropna=True)],
    "Total chunks":         [len(df)],
    "Secciones únicas":     [df["seccion"].nunique(dropna=True)],
})
display(summary)

# 5.b. Chunks por 'fuente'
chunks_per_fallo = (
    df.groupby("fuente")
      .size()
      .reset_index(name="n_chunks")
      .sort_values("n_chunks", ascending=False)
)
print("\n=== Chunks por fuente (fallo) ===")
display(chunks_per_fallo)

# 5.c. Chunks por 'seccion'
chunks_per_seccion = (
    df.groupby("seccion")
      .size()
      .reset_index(name="n_chunks")
      .sort_values("n_chunks", ascending=False)
)
print("\n=== Chunks por sección ===")
display(chunks_per_seccion)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Total de vectores en FAISS index: 4918
Total de Document en docstore: 4918

--- Primeros 5 Documentos ---
ID: 6c42f9cd-a447-40c4-b3c8-cc7b383a4a69
Texto (fragmento): '"URBANOTEC S.A. S/ QUIEBRA" - Expte. Nº 8985'…
Metadatos: {}
----------------------------------------
ID: 3d97289c-d85d-4a48-9c57-e9340a4457a5
Texto (fragmento): 'En la ciudad de Paraná, capital de la provincia de Entre Ríos, a los veinticinco días del mes de mar'…
Metadatos: {}
----------------------------------------
ID: 0dc1a4e6-b40c-4b06-b020-1380f459886d
Texto (fragmento): '1.- La sentencia de Camara que viene recurrida La sentencia de la Cámara de Apelaciones Sala Primera'…
Metadatos: {}
----------------------------------------
ID: ea826322-316b-4b29-a862-28370371fe45
Texto (fragmento): 'En consecuencia redujo los honorarios de los letrados Angelini y Bargas a la suma de pesos quiniento'…
Metadatos: {}
----------------------------------------
ID: 853f57c5-2dfb-4d1e-9ea2-bd847ffd569f
Texto (fragmento): 'Para así deci

,Total fallos únicos,Total chunks,Secciones únicas
0,0,4918,0



=== Chunks por fuente (fallo) ===


,fuente,n_chunks



=== Chunks por sección ===


,seccion,n_chunks


In [ ]:
if EMBEDDINGS_OPTION == "langchain":
    embedding_model = LangchainSentenceTransformer(model_name = EMBEDDINGS_MODEL)
    
elif EMBEDDINGS_OPTION == "azure_openai":
    embedding_model = AzureOpenAIEmbedder(
        deployment_name = EMBEDDINGS_DEPLOYMENT,
        endpoint        = EMBEDDINGS_ENDPOINT,
        api_key         = EMBEDDINGS_API_KEY,
        api_version     = EMBEDDINGS_VERSION,
    )


vectorstore = FAISS.load_local(
    "datasets/fallos_vectorstore_ada-002", 
    embedding_model,
    allow_dangerous_deserialization=True
)


def create_context(query):
    results = vectorstore.similarity_search(query, k=5)
    # for i, doc in enumerate(results, 1):
    #     print(f"{i}. {doc.page_content.strip()}\n")
    return results

azure_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,
    api_key=API_KEY
)

async def get_response(query):
    res = await azure_client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": "Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. Tu tarea es analizar el contexto y responder de manera precisa y concisa.",
            },
            {
                "role": "system", 
                "content": f"El contexto es: {create_context(query)}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        
    )
    return res

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [38]:
response = await get_response("Explicame el fallo que dice: TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS -  Expte. Nº 8142")

In [39]:
respuesta = response.choices[0].message.content
display(Markdown(respuesta))

El fallo correspondiente al expediente "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS" se refiere a un proceso judicial en el cual Alcides Vicente Terenzano demanda a Marta Olga Landivar y a otro demandado por un cobro de pesos. 

El tribunal considera la solicitud de regulación de honorarios presentada el 7 de diciembre de 2023, relacionada con la actuación profesional en recursos de inaplicabilidad de ley que fueron resueltos en fechas anteriores (25 de junio de 2020 y 10 de abril de 2023). Para este proceso, se toma en cuenta la normativa arancelaria aplicable, citando varios artículos de la ley 7046 que regulan tales situaciones.

En resumen, el fallo se centra en la regulación de honorarios por el trabajo realizado en los mencionados recursos, ajustándose a la legislación pertinente y la liquidación económica aprobada previamente.

In [40]:
def analyze_similarity_results(query, k=5):
    """
    Analiza y muestra los resultados de similitud para una consulta
    """
    print(f"🔍 Consulta: {query}")
    print("=" * 80)
    
    # Obtener resultados con scores de similitud
    if hasattr(vectorstore, 'similarity_search_with_score'):
        results_with_scores = vectorstore.similarity_search_with_score(query, k=k)
        
        print(f"\n📊 Top {k} documentos más similares:\n")
        
        for i, (doc, score) in enumerate(results_with_scores, 1):
            print(f"🏆 Resultado #{i} - Score: {score:.4f}")
            print(f"📁 Fuente: {doc.metadata.get('fuente', 'N/A')}")
            print(f"📋 Sección: {doc.metadata.get('seccion', 'N/A')}")
            print(f"📄 Contenido: {doc.page_content[:300]}...")
            print("-" * 80)
            
        return results_with_scores
    else:
        # Fallback si no tiene similarity_search_with_score
        results = vectorstore.similarity_search(query, k=k)
        
        print(f"\n📊 Top {k} documentos más similares:\n")
        
        for i, doc in enumerate(results, 1):
            print(f"🏆 Resultado #{i}")
            print(f"📁 Fuente: {doc.metadata.get('fuente', 'N/A')}")
            print(f"📋 Sección: {doc.metadata.get('seccion', 'N/A')}")
            print(f"📄 Contenido: {doc.page_content[:300]}...")
            print("-" * 80)
            
        return results

# Ejemplo de uso
query_test = "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS"
similarity_results = analyze_similarity_results(query_test, k=5)

🔍 Consulta: TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS

📊 Top 5 documentos más similares:

🏆 Resultado #1 - Score: 0.1322
📁 Fuente: N/A
📋 Sección: N/A
📄 Contenido: "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS"- Expte. Nº 8142.
PARANÁ, 5 de febrero de 2024....
--------------------------------------------------------------------------------
🏆 Resultado #2 - Score: 0.2176
📁 Fuente: N/A
📋 Sección: N/A
📄 Contenido: Que los autos "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS" - Expte. Nº 8142, vienen a consideración de este Tribunal en virtud de la solicitud de regulación de honorarios de fecha 7/12/2023, por la actuación profesional en los recursos de inaplicabilidad de...
--------------------------------------------------------------------------------
🏆 Resultado #3 - Score: 0.2483
📁 Fuente: N/A
📋 Sección: N/A
📄 Contenido: "PÉREZ, Eusebio Antonio C/ ZAMPEDRI, Adrián

In [41]:
def create_context_with_metadata(query, k=5):
    """
    Crea contexto incluyendo metadatos para mejor trazabilidad
    """
    results = vectorstore.similarity_search(query, k=k)
    
    context_parts = []
    for i, doc in enumerate(results, 1):
        fuente = doc.metadata.get('fuente', 'N/A')
        seccion = doc.metadata.get('seccion', 'N/A')
        
        context_part = f"""
Documento {i}:
Fuente: {fuente}
Sección: {seccion}
Contenido: {doc.page_content.strip()}
"""
        context_parts.append(context_part)
    
    return "\n".join(context_parts), results

In [42]:
# Configuración para evaluación ELO
class RAGEvaluator:
    def __init__(self, vectorstore, azure_client, deployment):
        self.vectorstore = vectorstore
        self.azure_client = azure_client
        self.deployment = deployment
        
    async def get_rag_response(self, query, k=5, system_prompt=None):
        """
        Obtiene respuesta RAG con contexto recuperado
        """
        context, retrieved_docs = create_context_with_metadata(query, k=k)
        
        if system_prompt is None:
            system_prompt = """Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. 
            Tu tarea es analizar el contexto proporcionado y responder de manera precisa y concisa, citando las fuentes cuando sea relevante."""
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "system", "content": f"Contexto de documentos relevantes:\n{context}"},
            {"role": "user", "content": query}
        ]
        
        response = await self.azure_client.chat.completions.create(
            model=self.deployment,
            messages=messages,
            temperature=0.1  # Baja temperatura para consistencia
        )
        
        return {
            'response': response.choices[0].message.content,
            'retrieved_docs': retrieved_docs,
            'context': context
        }
    
    async def evaluate_response_quality(self, query, response_a, response_b):
        """
        Evalúa qué respuesta es mejor usando el modelo como juez
        """
        evaluation_prompt = f"""
Eres un juez experto evaluando respuestas de un sistema de preguntas y respuestas sobre fallos judiciales.

Pregunta: {query}

Respuesta A: {response_a}

Respuesta B: {response_b}

Evalúa qué respuesta es mejor considerando:
1. Precisión factual
2. Relevancia al caso específico
3. Claridad y estructura
4. Uso apropiado del contexto legal

Responde únicamente con: "A", "B", o "EMPATE"
"""
        
        response = await self.azure_client.chat.completions.create(
            model=self.deployment,
            messages=[{"role": "user", "content": evaluation_prompt}],
            temperature=0.0
        )
        
        result = response.choices[0].message.content.strip().upper()
        return result if result in ["A", "B", "EMPATE"] else "EMPATE"

# Crear evaluador
evaluator = RAGEvaluator(vectorstore, azure_client, DEPLOYMENT)

In [43]:
import math
from collections import defaultdict

class ELOSystem:
    def __init__(self, k_factor=32, initial_rating=1500):
        self.k_factor = k_factor
        self.initial_rating = initial_rating
        self.ratings = defaultdict(lambda: initial_rating)
        self.match_history = []
    
    def expected_score(self, rating_a, rating_b):
        """Calcula la probabilidad esperada de que A gane contra B"""
        return 1 / (1 + 10**((rating_b - rating_a) / 400))
    
    def update_ratings(self, player_a, player_b, result):
        """
        Actualiza ratings basado en el resultado
        result: 1 si A gana, 0 si B gana, 0.5 si empate
        """
        rating_a = self.ratings[player_a]
        rating_b = self.ratings[player_b]
        
        expected_a = self.expected_score(rating_a, rating_b)
        expected_b = self.expected_score(rating_b, rating_a)
        
        new_rating_a = rating_a + self.k_factor * (result - expected_a)
        new_rating_b = rating_b + self.k_factor * ((1 - result) - expected_b)
        
        self.ratings[player_a] = new_rating_a
        self.ratings[player_b] = new_rating_b
        
        # Guardar historial
        self.match_history.append({
            'player_a': player_a,
            'player_b': player_b,
            'result': result,
            'rating_a_before': rating_a,
            'rating_b_before': rating_b,
            'rating_a_after': new_rating_a,
            'rating_b_after': new_rating_b
        })
        
        return new_rating_a, new_rating_b
    
    def get_leaderboard(self):
        """Retorna leaderboard ordenado por rating"""
        return sorted(self.ratings.items(), key=lambda x: x[1], reverse=True)

# Inicializar sistema ELO
elo_system = ELOSystem()

In [44]:
async def run_elo_evaluation(queries, configurations):
    """
    Ejecuta evaluación ELO entre diferentes configuraciones de RAG
    
    queries: Lista de preguntas de prueba
    configurations: Dict con diferentes configuraciones {name: config_dict}
    """
    
    print("🎯 Iniciando evaluación ELO...")
    
    # Generar todas las respuestas
    all_responses = {}
    
    for config_name, config in configurations.items():
        print(f"📝 Generando respuestas para configuración: {config_name}")
        all_responses[config_name] = {}
        
        for query in tqdm(queries, desc=f"Procesando {config_name}"):
            try:
                result = await evaluator.get_rag_response(
                    query, 
                    k=config.get('k', 5),
                    system_prompt=config.get('system_prompt', None)
                )
                all_responses[config_name][query] = result['response']
            except Exception as e:
                print(f"❌ Error en {config_name} para query '{query[:50]}...': {e}")
                all_responses[config_name][query] = "Error en generación"
    
    # Realizar comparaciones ELO
    print("\n⚔️ Realizando comparaciones ELO...")
    
    config_names = list(configurations.keys())
    total_comparisons = 0
    
    for i, config_a in enumerate(config_names):
        for j, config_b in enumerate(config_names[i+1:], i+1):
            print(f"\n🥊 {config_a} vs {config_b}")
            
            for query in tqdm(queries, desc="Comparando"):
                response_a = all_responses[config_a][query]
                response_b = all_responses[config_b][query]
                
                if "Error" in response_a or "Error" in response_b:
                    continue
                
                try:
                    judgment = await evaluator.evaluate_response_quality(
                        query, response_a, response_b
                    )
                    
                    # Convertir juicio a score ELO
                    if judgment == "A":
                        result = 1.0
                    elif judgment == "B":
                        result = 0.0
                    else:  # EMPATE
                        result = 0.5
                    
                    elo_system.update_ratings(config_a, config_b, result)
                    total_comparisons += 1
                    
                except Exception as e:
                    print(f"❌ Error en evaluación: {e}")
    
    print(f"\n✅ Evaluación completada. Total de comparaciones: {total_comparisons}")
    
    # Mostrar resultados
    print("\n🏆 RESULTADOS FINALES:")
    print("=" * 50)
    
    leaderboard = elo_system.get_leaderboard()
    for rank, (config_name, rating) in enumerate(leaderboard, 1):
        print(f"{rank}. {config_name}: {rating:.1f} ELO")
    
    return elo_system, all_responses

# Ejemplo de configuraciones para probar
configurations = {
    "RAG_Standard": {
        "k": 5,
        "system_prompt": """Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. 
        Tu tarea es analizar el contexto proporcionado y responder de manera precisa y concisa."""
    },
    "RAG_Detailed": {
        "k": 7,
        "system_prompt": """Eres un experto en derecho que analiza fallos judiciales. Proporciona respuestas detalladas, 
        cita las fuentes específicas cuando sea posible, y explica el razonamiento jurídico detrás de las decisiones."""
    },
    "RAG_Concise": {
        "k": 3,
        "system_prompt": """Eres un asistente legal que proporciona respuestas concisas y directas sobre fallos judiciales. 
        Enfócate en los puntos clave sin información redundante."""
    }
}

# Queries de prueba (agregar más según tus casos de uso)
test_queries = [
    "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS -  Expte. Nº 8142",
    "¿Cuáles son los fundamentos principales en casos de cobro de pesos?",
    "¿Qué criterios se usan para determinar la responsabilidad en casos civiles?",
    # Agregar más queries relevantes a tu dominio
]

In [45]:
# Ejecutar evaluación ELO
elo_results, response_cache = await run_elo_evaluation(test_queries, configurations)

# Análisis adicional
print("\n📊 ANÁLISIS DETALLADO:")
print("=" * 50)

# Historial de matches
matches_df = pd.DataFrame(elo_results.match_history)
print(f"Total de enfrentamientos: {len(matches_df)}")

# Estadísticas por configuración
for config_name in configurations.keys():
    wins = len(matches_df[
        ((matches_df['player_a'] == config_name) & (matches_df['result'] == 1.0)) |
        ((matches_df['player_b'] == config_name) & (matches_df['result'] == 0.0))
    ])
    losses = len(matches_df[
        ((matches_df['player_a'] == config_name) & (matches_df['result'] == 0.0)) |
        ((matches_df['player_b'] == config_name) & (matches_df['result'] == 1.0))
    ])
    ties = len(matches_df[
        ((matches_df['player_a'] == config_name) | (matches_df['player_b'] == config_name)) &
        (matches_df['result'] == 0.5)
    ])
    
    total_games = wins + losses + ties
    if total_games > 0:
        win_rate = wins / total_games * 100
        print(f"{config_name}: {wins}W-{losses}L-{ties}T (WR: {win_rate:.1f}%)")

🎯 Iniciando evaluación ELO...
📝 Generando respuestas para configuración: RAG_Standard


Procesando RAG_Standard: 100%|██████████| 3/3 [00:09<00:00,  3.32s/it]


📝 Generando respuestas para configuración: RAG_Detailed


Procesando RAG_Detailed: 100%|██████████| 3/3 [00:44<00:00, 14.85s/it]


📝 Generando respuestas para configuración: RAG_Concise


...


🥊 RAG_Standard vs RAG_Concise


Comparando: 100%|██████████| 3/3 [00:01<00:00,  1.98it/s]



🥊 RAG_Detailed vs RAG_Concise


Comparando: 100%|██████████| 3/3 [00:01<00:00,  1.69it/s]


✅ Evaluación completada. Total de comparaciones: 9

🏆 RESULTADOS FINALES:
1. RAG_Detailed: 1579.4 ELO
2. RAG_Standard: 1479.0 ELO
3. RAG_Concise: 1441.6 ELO

📊 ANÁLISIS DETALLADO:
Total de enfrentamientos: 9
RAG_Standard: 2W-4L-0T (WR: 33.3%)
RAG_Detailed: 6W-0L-0T (WR: 100.0%)
RAG_Concise: 1W-5L-0T (WR: 16.7%)
